In [1]:
import git
g = git.cmd.Git('/root/Projects/Assignment-Projects')
g.pull()

'Already up-to-date.'

In [1]:
from Sparker.SparkLogProcesser.SparkLogFileHandler import setSparkContext
from pyspark import SparkContext
sc = SparkContext() 
setSparkContext(sc)

In [2]:
from Sparker.Logic.LogicTests import *
day2Logs = readParsedLogsFromHDFS(entireDayParsedLogsFolder2)

hdfs://osldevptst01.host.gittigidiyor.net:8020/user/root/Parsed/2016-12-26_parsed has been read by 2017-05-21 22:54:16.985968


In [3]:
from MainSrc.PythonVersionHandler import *
from paths import *

In [4]:
from Sparker.Logic.LogicTests import *
from Sparker.Logic.Trainer import *
#trainPairWiseData(data) 

In [5]:
def printSeparater(c = 3):
    for n in range(c):
        print_('#' * 88)

In [6]:
from sklearn import svm

def evaluateModelOnData(model, data, dataName = 'Data', modelName = 'Model'):
    labelsAndPreds = data.map(lambda p: (p.label, model.predict(p.features)))
    truePredictionCount = labelsAndPreds.filter(lambda vp: vp[0] == vp[1]).count()
    instanceCount = data.count()
    accuracy = 100 * truePredictionCount / float(instanceCount)
    print_('\n'+modelName, 'has been evaluated on', dataName, 'by', nowStr())
    print_('The result accuracy is %' + '%.3f\n' % (accuracy))
    return labelsAndPreds

def trainPairWiseData(data, dataName = 'Data', modelName = 'Model', evaluate = True):
    print_('Training with SVMWithSGD...')
    model = SVMWithSGD.train(data, iterations=100)
    print_('\n', modelName, 'has been trained on', dataName, 'by', nowStr())##
    print_('The learned weights:\n' + str(model.weights).replace(',', ', ') + '\n')
    if evaluate:
        evaluateModelOnData(model, data, dataName, modelName)
    return model

def runTrainingExperiment(trainData, testData, modelName = 'Model', save = True, outputFolder = Day1_iPhone_6_DataFolder):
    model = trainPairWiseData(trainData, 'trainData', modelName)
    if save:
        modelPath = joinPath(outputFolder, modelName)
        try:
            model.save(sc_(), modelPath)
        except Py4JJavaError:
            pass
        print_(modelPath, 'has been saved successfully by', nowStr())
    return evaluateModelOnData(model, testData, 'testData', modelName)

In [7]:
def evaluateModelOnDataSK(model, data, dataName = 'Data', modelName = 'Model'):
    labelsAndPreds = data.map(lambda p: (p.label, model.predict(p.features)))
    truePredictionCount = labelsAndPreds.filter(lambda vp: vp[0] == vp[1]).count()
    instanceCount = data.count()
    accuracy = 100 * truePredictionCount / float(instanceCount)
    print_('\n'+modelName, 'has been evaluated on', dataName, 'by', nowStr())
    print_('The result accuracy is %' + '%.3f\n' % (accuracy))
    return labelsAndPreds

def trainPairWiseDataSK(data, dataName = 'Data', modelName = 'Model', evaluate = True):
    model = svm.SVC()#kernel =  'poly', degree = 3) #[:size][:size]
    size = 5000
    X =  data.map(lambda p: p.features).collect()
    Y =  data.map(lambda p: p.label).collect()
    print_('Training with SVC...')
    model.fit(X, Y)
    print_('\n', modelName, 'has been trained on', dataName, 'by', nowStr())##
   # print_('The learned weights:\n' + str(model.coef_).replace(',', ', ') + '\n')
    if evaluate:
        evaluateModelOnDataSK(model, data, dataName, modelName)
    return model

def runTrainingExperimentSK(trainData, testData, modelName = 'Model', save = True, outputFolder = Day1_iPhone_6_DataFolder):
    model = trainPairWiseDataSK(trainData, 'trainData', modelName)
    if save:
        modelPath = joinPath(outputFolder, modelName)
        try:
            model.save(sc_(), modelPath)
        except Py4JJavaError:
            pass
        print_(modelPath, 'has been saved successfully by', nowStr())
    return evaluateModelOnDataSK(model, testData, 'testData', modelName)

In [18]:
from DeepLearningToRank.DeepDataHandler import readTrainDataFromPickle
#from DeepLearningToRank.DeepTrainer import *

def featureFilter(trainData):
    return trainData.map(lambda x: LabeledPoint(x.label, list(x.features[1:4]) + list(x.features[5:6]) + list(x.features[7:-2])))

def runExperiment(trainData, modelName = 'Model'):
    printSeparater(1)
    testData, trainData = trainData.randomSplit(weights=[0.3, 0.7])
    #testData, trainData = trainData[:int(30*len(trainData)/100)], trainData[int(30*len(trainData)/100):]SKSK
    runTrainingExperiment(trainData, testData, modelName = modelName, save = False)
    print_('The experiment for', modelName, 'has finished by', nowStr())
    
def trainPairWiseDataTestKeyword2(keyword, location = 'HDFS'):
    printSeparater(2)
    keyword = keyword.replace(' ', '_')
    inputName = 'all_day'
    if location == 'HDFS':
        outputFolder = joinPath(HDFSDataFolder, 'Day1_' + keyword + '_Data')
        #outputFolder = joinPath(joinPath(textTrainDataFolder, 'HDFS'), 'Day1_' + keyword + '_Data')
        trainDataFile = joinPath(outputFolder, inputName + '_' + keyword + '_TrainData_Updated')
        trainData = readTrainDataFromHDFS(trainDataFile)
    elif location == 'oneHot':
        outputFolder = joinPath(joinPath(textTrainDataFolder, 'HDFS'), 'Day1_' + keyword + '_Data')
        outputFolder = joinPath(joinPath(textTrainDataFolder, 'HDFS'), 'Day1_' + keyword + '_Data')
        trainDataFile = joinPath(outputFolder, inputName + '_' + keyword + '_OneHot_TrainData.txt')
        trainData = readTrainDataFromPickle(trainDataFile)
        trainData = sc_().parallelize(trainData)
        trainData = trainData.map(lambda x: LabeledPoint(x[0], x[1]))
    else:
        trainDataFile = joinPath(textTrainDataFolder, inputName + '_' + keyword + '_TrainData.txt')
        trainData = readTrainDataFromPickle(trainDataFile)
        trainData = sc_().parallelize(trainData)
        trainData = trainData.map(lambda x: LabeledPoint(x[0], x[1]))
    print_(trainData.first())
    print_('Data from', location)
    preprocessing = scaleTrainData# normalizeTrainData#
    trainData = preprocessing(trainData)
    runExperiment(trainData, modelName = 'ExtendedModel')
    trainData = featureFilter(trainData)
    runExperiment(trainData, modelName = 'FilteredModel')
    printSeparater(2)

In [19]:
#trainPairWiseDataTestKeyword2('iphone 6', 'oneHot')
trainPairWiseDataTestKeyword2('tv_unitesi', 'HDFS')

########################################################################################
########################################################################################
hdfs://osldevptst01.host.gittigidiyor.net:8020/user/root/data/Day1_tv_unitesi_Data/all_day_tv_unitesi_TrainData_Updated has been read successfully by 2017-05-21 23:04:33.174900
(1.0,[1.0,0.0,6.0,-5204.0,1.0,1.0,0.0,0.0,0.0,1.0,-82.5,1.0])
Data from HDFS
3092 instances have been scaled by 2017-05-21 23:04:33.582363
########################################################################################
Training with SVMWithSGD...

 ExtendedModel has been trained on trainData by 2017-05-21 23:04:39.640349
The learned weights:
[-0.513755623374, 0.986178164193, 0.000779970943234, 0.253640344046, -0.0857450149173, -0.0761931493792, 0.0950914633394, 0.0525474647189, 0.332135161778, 0.133899960777, -0.556635951003, 0.402134689711]


ExtendedModel has been evaluated on trainData by 2017-05-21 23:04:40.504959
The result

In [13]:
import pickle
def savePairWisetrainData(keyword):
    printSeparater(2)
    keyword = keyword.replace(' ', '_')
    inputName = 'all_day'
    outputFolder = joinPath(HDFSDataFolder, 'Day1_' + keyword + '_Data')
    dataTypes = ['_TrainData', '_labeledPairs', '_journey_products']
    for typ in dataTypes:
        fileName = inputName + '_' + keyword + typ
        trainDataFile = joinPath(outputFolder, fileName)
        trainData = sc_().textFile(trainDataFile)
        f = joinPath('/root/Projects/TrainData6', fileName)
        trainData = trainData.collect()
        fp = open(f, "wb")   #Pickling
        pickle.dump(trainData, fp)
        print_(f, 'has been saved successfully by', nowStr())

In [21]:
def trainPairWiseDataTest():
    keywords = ['iphone 6', 'nike air max', 'tv unitesi', 'camasir makinesi', 'bosch', 'jant', 'kot ceket']
    for keyword in keywords[:4]:
        trainPairWiseDataTestKeyword2(keyword)
        #savePairWisetrainData(keyword)readTrainDataFromHDFS

In [22]:
def main():
    printSeparater()
    print_('%s:' % nowStr(), 'Running on osldevptst02:Jupyter...')

    trainPairWiseDataTest()

    print_('%s:' % nowStr(), 'DONE')
    printSeparater()


In [23]:
main()

########################################################################################
########################################################################################
########################################################################################
2017-05-21 23:08:16.204199: Running on osldevptst02:Jupyter...
########################################################################################
########################################################################################
hdfs://osldevptst01.host.gittigidiyor.net:8020/user/root/data/Day1_iphone_6_Data/all_day_iphone_6_TrainData_Updated has been read successfully by 2017-05-21 23:08:16.276956
(0.0,[-7.0,-1.0,-67.0,-64.0,0.0,0.0,0.0,-1.0,0.0,0.0,493.9,0.0])
Data from HDFS
350 instances have been scaled by 2017-05-21 23:08:16.959146
########################################################################################
Training with SVMWithSGD...

 ExtendedModel has been trained on trainData by 2017-05-21 2